Week 2 Peer Graded Assessment (Battle of Neighbourhoods)

Note to Peers: I have completed the entire assignment for week 3 in this single notebook. I will provide the link to this notebook in all the required fields. Please evaluate accordingly.

# Part 1: Data wrangling and pre-processing

In [1]:
import numpy as np
import pandas as pd
import json
from sklearn.cluster import KMeans
import folium
import requests
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize

In [2]:
source = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(source)

result.headers

{'Date': 'Fri, 15 May 2020 14:33:44 GMT', 'Vary': 'Accept-Encoding,Cookie,Authorization', 'Server': 'ATS/8.0.7', 'Content-Type': 'text/html; charset=UTF-8', 'X-Content-Type-Options': 'nosniff', 'P3P': 'CP="See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-language': 'en', 'Last-Modified': 'Thu, 07 May 2020 17:47:29 GMT', 'Content-Encoding': 'gzip', 'Age': '52159', 'X-Cache': 'cp5008 miss, cp5007 hit/63', 'X-Cache-Status': 'hit-front', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Set-Cookie': 'WMF-Last-Access=16-May-2020;Path=/;HttpOnly;secure;Expires=Wed, 17 Jun 2020 00:00:00 GMT, WMF-Last-Access-Global=16-May-2020;Path=/;Domain=.wikipedia.org;HttpOnly;secure;Expires=Wed, 17 Jun 2020 00:00:00 GMT, GeoIP=IN:DL:New_Delhi:28.63:77.22:v4; Path=/; secure; Domain=.wikipedia.org', 'X-Client-IP': '125.99.211.184', 'Cache-Control': 'private, s-maxage=0, max-age=0, must-revalidate

In [4]:
from bs4 import BeautifulSoup
bs = BeautifulSoup(result.content, 'html.parser')
table = bs.find('table')
trs = table.find_all('tr')
rows = []

for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)

lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])
        
lst        

[['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront'],
 ['M6A', 'North York', 'Lawrence Manor, Lawrence Heights'],
 ['M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government"],
 ['M9A', 'Etobicoke', 'Islington Avenue'],
 ['M1B', 'Scarborough', 'Malvern, Rouge'],
 ['M3B', 'North York', 'Don Mills'],
 ['M4B', 'East York', 'Parkview Hill, Woodbine Gardens'],
 ['M5B', 'Downtown Toronto', 'Garden District, Ryerson'],
 ['M6B', 'North York', 'Glencairn'],
 ['M9B',
  'Etobicoke',
  'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale'],
 ['M1C', 'Scarborough', 'Rouge Hill, Port Union, Highland Creek'],
 ['M3C', 'North York', 'Don Mills'],
 ['M4C', 'East York', 'Woodbine Heights'],
 ['M5C', 'Downtown Toronto', 'St. James Town'],
 ['M6C', 'York', 'Humewood-Cedarvale'],
 ['M9C',
  'Etobicoke',
  'Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood'],
 ['M1E', 'Scarborou

Finally, printing the shape of the Data Frame and checking if merging  of neighborhoods was successful

In [12]:
cols = ['Postal Code', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns = cols)
print(df.shape)
df.loc[df['Postal Code'] == 'M5A']

(103, 3)


,Postal Code,Borough,Neighborhood
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# Part 2: Using csv file to add co-ordinates to the Data Frame

In [15]:
df_coor = pd.read_csv(r"C:\Users\Raevant\Downloads\Geospatial_Coordinates.csv")
df_fin = pd.merge(df, df_coor, on = "Postal Code", how = 'left')
df_fin.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Part 3: Toronto Map, Cluster Analysis

In [17]:
geolocator = Nominatim(user_agent = "coursera")
address = 'Toronto'

try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))
    
tor_map = folium.Map(location = [latitude, longitude], zoom_start = 11)

for lat, lon, label in zip(df_fin['Latitude'], df_fin['Longitude'], df_fin['Postal Code']):
    label = folium.Popup(label)
    folium.CircleMarker([lat, lon], raduis = 2, popup = label, color = 'blue', fill_color='#3186cc',
        fill_opacity=0.7).add_to(tor_map)
    
tor_map    

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.
